Finetuning different pretrained BERTs on MedNLI dataset
================

Notebook adapted from https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html

MedNLI has 3 labels: entailment, contradiction, and neutral




## Setup

In [7]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import json
import re
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from bert import data, model



np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0) 
#ctx = mx.cpu()

### Get Pretrained Bert

In [8]:
bert_models = ['book_corpus_wiki_en_cased',
               'biobert_v1.1_pubmed_cased',
              'clinicalbert_uncased']

In [9]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name=bert_models[1],
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0.1, axes=())
          )
          (proj_query): Dense(768 -> 768, linear)
          (proj_key): Dense(768 -> 768, linear)
          (proj_value): Dense(768 -> 768, linear)
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): BERTPositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (la

### Transform the model for SentencePair Classification
by adding a BERTClassifier and a dense layer
specify softmax for cross entropy

In [10]:
model = model.classification.BERTClassifier(bert_base, num_classes=3, dropout=0.1)
# only need to initialize the classifier layer.
model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
model.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

### Transform original MedNLI jsonl files to TSV files

In [11]:
def get_tokens(sentence_binary_parse):
    sentence = sentence_binary_parse \
        .replace('(', ' ').replace(')', ' ') \
        .replace('-LRB-', '(').replace('-RRB-', ')') \
        .replace('-LSB-', '[').replace('-RSB-', ']')

    tokens = sentence.split()

    return tokens


def toTSV_mednli(filename,output):
    

    label_dict = {'entailment':'0',
                 'contradiction':'1',
                 'neutral':'2'}
    
    with open(filename, 'r') as f:
        with open(output,'w') as o:
            for i,line in enumerate(f): #- add index for tracking which sentence
                example = json.loads(line)

                premise = ' '.join( i for i in get_tokens(example['sentence1_binary_parse']))
                hypothesis = ' '.join( i for i in get_tokens(example['sentence2_binary_parse']))

                label = label_dict[example.get('gold_label', None)]

                o.write(f"{label}\t{premise}\t{hypothesis}\n")



toTSV_mednli('mli_test_v1.jsonl','mli_test.tsv')
toTSV_mednli('mli_train_v1.jsonl','mli_train.tsv')
toTSV_mednli('mli_dev_v1.jsonl','mli_dev.tsv')


In [6]:
# print out some pairs
test_tsv = io.open('mli_test.tsv', encoding='utf-8')
for i in range(3):
    print(test_tsv.readline())

0	In the ED , initial VS revealed T 98.9 , HR 73 , BP 121/90 , RR 15 , O2 sat 98 % on RA .	The patient is hemodynamically stable

1	In the ED , initial VS revealed T 98.9 , HR 73 , BP 121/90 , RR 15 , O2 sat 98 % on RA .	The patient is hemodynamically unstable .

2	In the ED , initial VS revealed T 98.9 , HR 73 , BP 121/90 , RR 15 , O2 sat 98 % on RA .	The patient is in pain .



### Load the Dataset in TSV format

In [98]:
# Skip the first line, which is the schema
num_discard_samples = 0
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [1, 2, 0]
data_train_raw = nlp.data.TSVDataset(filename='mli_train.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)


data_train_raw = nlp.data.TSVDataset(filename='mli_train.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)

data_dev_raw = nlp.data.TSVDataset(filename='mli_dev.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)

data_test_raw = nlp.data.TSVDataset(filename='mli_test.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)


sample_id = 8
# Sentence A
print(data_train_raw[sample_id][0])
# Sentence B
print(data_train_raw[sample_id][1])
# 0 means entailment, 1 contradiction, 2 neutral
print(data_train_raw[sample_id][2])

The patient was seen by his primary care physician after he had complained of a one-week history of dyspnea on exertion and jaw tightness .
The patient has has coronary artery disease .
2


### Preprocess the data to BERT format

We will use
`BERTDatasetTransform` to perform the following transformations:
- tokenize
the
input sequences
- insert [CLS] at the beginning
- insert [SEP] between sentence
A and sentence B, and at the end
- generate segment ids to indicate whether
a token belongs to the first sequence or the second sequence.
- generate valid length

In [99]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = similar)]
all_labels = ["0", "1","2"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)
data_dev = data_dev_raw.transform(transform)
data_test = data_test_raw.transform(transform)


print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('valid length = \n%s'%data_train[sample_id][1])
print('segment ids = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

vocabulary used for tokenization = 
Vocab(size=28996, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 0
[CLS] token id = 101
[SEP] token id = 102
token ids = 
[  101  1103  5351  1108  1562  1118  1117  2425  1920  7454  1170  1119
  1125 10790  1104   170  1141   118  1989  1607  1104   173  6834  1643
 25362  1113  4252  7340  1988  1105  5139  3600  1757   119   102  1103
  5351  1144  1144  1884 15789  1616 18593  3653   119   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
valid length = 
46
segment id

## Fine-tuning the model

Now we have all the pieces to put together, and we can finally start fine-tuning the
model with very few epochs. For demonstration, we use a fixed learning rate and
skip the validation steps. For the optimizer, we leverage the ADAM optimizer which
performs very well for NLP data and for BERT models in particular.

In [62]:
# The hyperparameters
batch_size = 16 # adjust this according to the GPU memory, too high batch size, out of memory will occur
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(model.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in model.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 16
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    y_hat = []
    y_label = []
    
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])
        
#         #copy to cpu
        
     
#         y_pred = out.copyto(mx.cpu()).asnumpy()
#         y_pred = [np.argmax(i) for i in y_pred]
#         y_hat.extend(y_pred)
#         y_true = label.copyto(mx.cpu()).asnumpy().T.flatten()
#         y_label.extend(y_true)
        
      
    
        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
    
#             print('\n')
#             print('ACCCCC')
#             print(np.mean(np.array(y_hat) == np.array(y_label)))
            step_loss = 0
            
#     print('\n\n\n\n')
#     print(f'{epoch_id} accuracy {np.mean(np.array(y_hat) == np.array(y_label))}')
    

[Epoch 0 Batch 16/706] loss=0.1349, lr=0.0000050, acc=0.965
[Epoch 0 Batch 32/706] loss=0.1525, lr=0.0000050, acc=0.960
[Epoch 0 Batch 48/706] loss=0.0926, lr=0.0000050, acc=0.966
[Epoch 0 Batch 64/706] loss=0.1593, lr=0.0000050, acc=0.963
[Epoch 0 Batch 80/706] loss=0.1569, lr=0.0000050, acc=0.964
[Epoch 0 Batch 96/706] loss=0.1101, lr=0.0000050, acc=0.964
[Epoch 0 Batch 112/706] loss=0.2182, lr=0.0000050, acc=0.960
[Epoch 0 Batch 128/706] loss=0.1087, lr=0.0000050, acc=0.961
[Epoch 0 Batch 144/706] loss=0.1755, lr=0.0000050, acc=0.960
[Epoch 0 Batch 160/706] loss=0.1262, lr=0.0000050, acc=0.961
[Epoch 0 Batch 176/706] loss=0.2048, lr=0.0000050, acc=0.960
[Epoch 0 Batch 192/706] loss=0.0949, lr=0.0000050, acc=0.962
[Epoch 0 Batch 208/706] loss=0.1179, lr=0.0000050, acc=0.961
[Epoch 0 Batch 224/706] loss=0.2011, lr=0.0000050, acc=0.961
[Epoch 0 Batch 240/706] loss=0.1660, lr=0.0000050, acc=0.961
[Epoch 0 Batch 256/706] loss=0.1397, lr=0.0000050, acc=0.960
[Epoch 0 Batch 272/706] loss=0

In [129]:
# GET DEV Accuracy
batch_size = 16 # adjust this according to the GPU memory, too high batch size, out of memory will occur
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_dev],
                                            batch_size=batch_size,
                                            shuffle=False)
bert_dataloader = mx.gluon.data.DataLoader(data_dev, batch_sampler=train_sampler)

# trainer = mx.gluon.Trainer(model.collect_params(), 'adam',
#                            {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in model.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 16
num_epochs = 1
for epoch_id in range(num_epochs):
    metric.reset()
    y_hat = []
    y_label = []
    
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
           
        metric.update([label], [out])
        
        #copy to cpu
        
     
        y_pred = out.copyto(mx.cpu()).asnumpy()
        y_pred = [np.argmax(i) for i in y_pred]
        y_hat.extend(y_pred)
        y_true = label.copyto(mx.cpu()).asnumpy().T.flatten()
        y_label.extend(y_true)
        
      
    
        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
        
            
    print('\n\n\n\n')
    print(f'DEV accuracy {np.mean(np.array(y_hat) == np.array(y_label))}')
    

[Epoch 0 Batch 16/91] loss=0.0000, lr=0.0000050, acc=0.825
[Epoch 0 Batch 32/91] loss=0.0000, lr=0.0000050, acc=0.814
[Epoch 0 Batch 48/91] loss=0.0000, lr=0.0000050, acc=0.812
[Epoch 0 Batch 64/91] loss=0.0000, lr=0.0000050, acc=0.828
[Epoch 0 Batch 80/91] loss=0.0000, lr=0.0000050, acc=0.827





DEV accuracy 0.8272401433691756


In [130]:
# GET TEST Accuracy
batch_size = 15 # adjust this according to the GPU memory, too high batch size, out of memory will occur
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
# train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_test],
#                                             batch_size=batch_size,
#                                             shuffle=False)
# bert_dataloader = mx.gluon.data.DataLoader(data_dev, batch_sampler=train_sampler)


bert_dataloader = mx.gluon.data.DataLoader(data_test, batch_size=15)
# trainer = mx.gluon.Trainer(model.collect_params(), 'adam',
#                            {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in model.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 15
num_epochs = 1
for epoch_id in range(num_epochs):
    metric.reset()
    y_hat = []
    y_label = []
    
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
           
        metric.update([label], [out])
        
        #copy to cpu
        
     
        y_pred = out.copyto(mx.cpu()).asnumpy()
        y_pred = [np.argmax(i) for i in y_pred]
        y_hat.extend(y_pred)
        y_true = label.copyto(mx.cpu()).asnumpy().T.flatten()
        y_label.extend(y_true)
        
      
      
    
        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
        
            
    print('\n\n\n\n')
    print(f'TEST accuracy {np.mean(np.array(y_hat) == np.array(y_label))}')

[Epoch 0 Batch 15/95] loss=0.0000, lr=0.0000050, acc=0.858
[Epoch 0 Batch 30/95] loss=0.0000, lr=0.0000050, acc=0.818
[Epoch 0 Batch 45/95] loss=0.0000, lr=0.0000050, acc=0.815
[Epoch 0 Batch 60/95] loss=0.0000, lr=0.0000050, acc=0.818
[Epoch 0 Batch 75/95] loss=0.0000, lr=0.0000050, acc=0.816
[Epoch 0 Batch 90/95] loss=0.0000, lr=0.0000050, acc=0.826





TEST accuracy 0.8241912798874824


In [131]:
import pandas as pd
df = pd.DataFrame({'y_hat':y_hat,'label':y_label})
df

,y_hat,label
0,0,0
1,1,1
2,2,2
3,0,0
4,1,1
5,1,2
6,2,0
7,1,1
8,2,2
9,0,0


In [132]:
df2 = pd.read_csv('mli_test.tsv',sep='\t',header=None)
df2.columns = ['class','premise','hypothesis']
df2.shape

(1422, 3)

In [137]:
df.merge(df2,how='inner',left_index=True, right_index=True).to_csv('bioberttest.csv',index=False)


In [133]:
df_final = df.merge(df2,how='inner',left_index=True, right_index=True)


In [136]:
np.mean(df_final.y_hat == df_final['class'])

0.8241912798874824